In [31]:
import os
# import platform
import sys
import pandas as pd
import numpy as np
import calendar
import datetime

In [39]:
workdir = '/Users/pnorton/Projects/Streamflow_CONUS/datapull_CONUS_20200123'
obsfile = f'{workdir}/conus_annual_HUC_10_WY_obs'
stnfile = f'{workdir}/conus_annual_HUC_10_WY_stn'

st = datetime.datetime(1960,10,1)
en = datetime.datetime(2018,9,30)

# Compute the period of record for this date range
por = en.year - st.year

In [40]:
def dparse(*dstr):
    dint = [int(x) for x in dstr]

    if len(dint) == 2:
        # For months we want the last day of each month
        dint.append(calendar.monthrange(*dint)[1])
    if len(dint) == 1:
        # For annual we want the last day of the year
        dint.append(12)
        dint.append(calendar.monthrange(*dint)[1])

    return datetime.datetime(*dint)

def dparse_wy(*dstr):
    # The water year version of dparse is only intended for working
    # with annual data. The monthly and daily dates are always
    # based on a calender year.
    dint = [int(x) for x in dstr]

    if len(dint) == 1:
        # For annual we want the last day of the year
        dint.append(9)
        dint.append(calendar.monthrange(*dint)[1])

    return datetime.datetime(*dint)

In [88]:
stn_col_names = ['site_no', 'station_nm', 'dec_lat_va', 'dec_long_va', 'drain_area_va', 'contrib_drain_area_va']
stn_col_types = [np.str_, np.str_, np.float_, np.float_, np.float_, np.float_]
stn_cols = dict(zip(stn_col_names, stn_col_types))

stations = pd.read_csv(stnfile, sep='\t', usecols=stn_col_names, dtype=stn_cols)
stations.set_index('site_no', inplace=True)

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Import the streamflow data. Use a custom date parser to create datetime values
# at the end of each water year.
obs_col_names = ['site_no', 'ts_id', 'year_nu', 'mean_va']
obs_col_types = [np.str_, np.int16, np.int16, np.float64]
obs_cols = dict(zip(obs_col_names, obs_col_types))

# working with observation dates (years) given in water years.
thedata = pd.read_csv(obsfile, sep='\t', usecols=obs_col_names, dtype=obs_cols,
                      date_parser=dparse_wy, parse_dates={'wyear': ['year_nu']},
                      index_col='wyear', keep_date_col=True)

# Select only the observations that are within our period of interest
thedata = thedata[(thedata.index >= st) & (thedata.index <= en)]

# Group by site_no and dd_nu. Filter by sites that don't have enough observations
# in the period of interest
thedata = thedata.groupby(['site_no', 'ts_id']).filter(lambda x: len(x) >= por)

# ------------------------------------------------------------------------
# Write out the annual observations
# thedata.reset_index().to_csv('%s_obs.tab' % args.outfile, sep='\t', index=False,
#                              header=['siteno', 'date', 'waterYr', 'avgQ'],
#                              float_format='%.3f',
#                              columns=['site_no', 'wyear', 'year_nu', 'mean_va'])

# Pivot the table so waterYr is the row index and each site is a column
sitedataByCol = thedata.reset_index().pivot(index='wyear', columns='site_no', values='mean_va')

In [89]:
stations.info()

<class 'pandas.core.frame.DataFrame'>
Index: 905 entries, 05018000 to 402114105350101
Data columns (total 5 columns):
station_nm               905 non-null object
dec_lat_va               905 non-null float64
dec_long_va              905 non-null float64
drain_area_va            857 non-null float64
contrib_drain_area_va    333 non-null float64
dtypes: float64(4), object(1)
memory usage: 42.4+ KB


In [90]:
sitedataByCol.head()

site_no,06025500,06038500,06052500,06054500,06066500,06089000,06090300,06090800,06101500,06108000,...,06912500,06913500,06916600,06917000,06919500,06921200,06921350,06932000,06933500,06934500
wyear,,,,,,,,,,,,,,,,,,,,,
1961-09-30,741.7,596.9,674.2,2927.0,3689.0,496.6,4684.0,4756.0,572.2,55.6,...,293.20,1304.0,3771.0,391.0,392.2,121.2,646.6,163.6,2864.0,79180.0
1962-09-30,1091.0,1001.0,1278.0,4898.0,4276.0,776.9,6144.0,6345.0,788.0,100.4,...,298.50,1193.0,3023.0,266.7,311.2,67.8,221.9,126.0,1797.0,84920.0
1963-09-30,1200.0,982.7,1175.0,5039.0,5369.0,453.9,6670.0,6918.0,529.4,75.9,...,77.50,207.8,475.5,26.2,100.9,55.1,67.8,81.6,1511.0,44980.0
1964-09-30,1369.0,1022.0,1211.0,5389.0,5771.0,1119.0,8473.0,9040.0,1243.0,350.2,...,4.44,116.6,550.9,53.6,113.0,25.9,163.1,91.6,991.2,47450.0
1965-09-30,1843.0,1143.0,1435.0,6479.0,6956.0,995.8,9514.0,10680.0,1193.0,235.6,...,142.10,798.6,2443.0,165.9,333.1,71.3,406.9,79.4,1752.0,80110.0


In [91]:
# df.groupby([df.index.year // 10 * 10, df.Company]).rolling('3650d').mean()

In [92]:
# sitedataByCol.rolling(window=10, min_periods=10).mean().pct_change(10)
lastten = sitedataByCol.last('10Y').mean().rename('last_ten_yr')

In [93]:
firstten = sitedataByCol.first('10Y').mean().rename('first_ten_yr')

In [94]:
pct_chg = ((lastten - firstten) / firstten).rename('pct_chg')

In [95]:
# .merge(xdf_df, left_on='hru_id_nat', right_index=True, how='left')
df = pd.concat([firstten, lastten, pct_chg], axis=1)

In [96]:
df

,first_ten_yr,last_ten_yr,pct_chg
site_no,,,
06025500,1234.38,1183.40,-0.041300
06038500,1055.06,986.77,-0.064726
06052500,1257.76,1082.20,-0.139581
06054500,5290.90,4897.10,-0.074430
06066500,5719.30,5224.30,-0.086549
...,...,...,...
06921200,80.49,113.68,0.412349
06921350,403.28,594.31,0.473691
06932000,123.92,192.05,0.549790


In [97]:
stations

,station_nm,dec_lat_va,dec_long_va,drain_area_va,contrib_drain_area_va
site_no,,,,,
05018000,St. Mary Canal at intake near Babb MT,48.852778,-113.416944,NaN,NaN
05018500,St. Mary Canal at St. Mary Crossing near Babb MT,48.946967,-113.375331,NaN,NaN
06006000,Red Rock Cr ab Lakes nr Lakeview MT,44.617031,-111.656747,37.30,NaN
06012500,Red Rock R bl Lima Reservoir nr Monida MT,44.655881,-112.371222,566.00,NaN
06016000,Beaverhead River at Barretts MT,45.116128,-112.750494,2730.00,NaN
...,...,...,...,...,...
394329104490101,"TOLL GATE CREEK ABOVE 6TH AVE AT AURORA, CO",39.725089,-104.817894,34.60,NaN
394839104570300,"SAND CREEK AT MOUTH NR COMMERCE CITY,CO",39.810972,-104.951583,187.00,NaN
401723105400000,ANDREWS CREEK-LOCH VALE-RMNP,40.290000,-105.666667,0.66,NaN


In [98]:
df.merge(stations, left_index=True, right_index=True, how='left')

,first_ten_yr,last_ten_yr,pct_chg,station_nm,dec_lat_va,dec_long_va,drain_area_va,contrib_drain_area_va
site_no,,,,,,,,
06025500,1234.38,1183.40,-0.041300,Big Hole River near Melrose MT,45.526581,-112.701725,2472.0,NaN
06038500,1055.06,986.77,-0.064726,Madison River bl Hebgen Lake nr Grayling MT,44.866392,-111.338781,931.0,NaN
06052500,1257.76,1082.20,-0.139581,Gallatin River at Logan MT,45.885356,-111.438286,1789.0,NaN
06054500,5290.90,4897.10,-0.074430,Missouri River at Toston MT,46.146572,-111.420278,14641.0,NaN
06066500,5719.30,5224.30,-0.086549,Missouri River bl Holter Dam nr Wolf Cr MT,46.994739,-112.010667,16924.0,NaN
...,...,...,...,...,...,...,...,...
06921200,80.49,113.68,0.412349,"Lindley Creek near Polk, MO",37.750472,-93.266194,112.0,NaN
06921350,403.28,594.31,0.473691,"Pomme de Terre River near Hermitage, MO",37.906056,-93.328917,615.0,NaN
06932000,123.92,192.05,0.549790,"Little Piney Creek at Newburg, MO",37.909528,-91.903333,200.0,NaN
